In [17]:
import numpy as np
import pandas as pd  
import time

In [18]:
dfUser = pd.read_csv('../data_csv/User.csv')
dfOrderDetail = pd.read_csv('../data_csv/OrderDetail.csv')
dfOrderDetail = dfOrderDetail.rename(columns= {"code" : 'code_order' , 'name' : 'name_order'})
dfCustomerDetail = pd.read_csv('../data_csv/CustomerDetail.csv')
dfCustomerDetail = dfCustomerDetail.drop(columns= {'id' , 'creator_id' , 'created_at' , 'updated_at' , 'deleted_at' , 'email' , 'phone' , 'fax' , 'tax_number' , 'address' , 'media_id' , 'user_id'})
dfCustomerDetail = dfCustomerDetail.rename(columns= {'code' : 'code_cus' , 'name' : 'name_cus' })
# dfCustomerDetail.head(2)
dfOrderDetail = dfOrderDetail.merge(dfCustomerDetail , on = 'customer_id' , how = 'left')
dfOrderDetail.head(2)


,code_order,name_order,qrcode,description,quantity,creator_id,created_at,updated_at,deleted_at,color,...,supplier_delivery_id,tax_id,payment_method_id,currency_id,debit_detail_id,is_complete,vnd_exchange_rate,vnd_exchange,code_cus,name_cus
0,DYC-US-220425-Ginè Energy-558,DYC-US-220425-Ginè Energy-558,1666975066DH32,<p>Đơn h&agrave;ng Reorder DYC-US-220219-Gin&e...,100.0,234,2022-05-05 17:05:01+00:00,2022-10-28 23:37:47+00:00,NaN,NaN,...,NaN,NaN,3.0,2,NaN,1,22800.0,9120000.0,USA,DESIGN YOUR CAPS USA LLC
1,DYC-US-220425-Vendome-557,DYC-US-220425-Vendome-557,1666975067DH33,NaN,100.0,234,2022-05-05 17:14:22+00:00,2022-10-28 23:37:47+00:00,NaN,NaN,...,NaN,NaN,3.0,2,NaN,1,22800.0,0.0,USA,DESIGN YOUR CAPS USA LLC


In [19]:
dfProductionTracking = pd.read_csv('../data_csv/ProductionTracking.csv')
dfProductionTrackingDetail = pd.read_csv('../data_csv/ProductionTrackingDetail.csv')
dfManufactureDeptDetail = pd.read_csv('../data_csv/ManufactureDeptDetail.csv')
dfProductionNotExistOrder = pd.read_csv('../data_csv/ProductionNotExistOrder.csv')

dfManufactureDeptDetail = dfManufactureDeptDetail.rename(columns= {'code' : 'code_dept' , 'name' : 'name_dept'})
dfProductionNotExistOrder = dfProductionNotExistOrder.rename(columns= {'id' : 'prod_not_exist_order_id','created_at' : 'not_exist_create' , 'name' : 'name_not_exist'}).drop(columns= {'updated_at' , 'deleted_at' , 'qrcode'})

In [20]:
dfProductionTracking = dfProductionTracking.merge(dfOrderDetail[['code_order' , 'name_order' , 'order_id' , 'name_cus']], on = 'order_id' , how = 'left')
dfProductionTracking = dfProductionTracking.merge(dfManufactureDeptDetail[['code_dept' , 'name_dept' , 'manufacture_dept_id']], on = 'manufacture_dept_id' , how = 'left')
dfProductionTracking = dfProductionTracking.merge(dfProductionNotExistOrder, on = 'prod_not_exist_order_id' , how = 'left')

In [21]:
dfProductionTracking.head(2)

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,code_order,name_order,name_cus,code_dept,name_dept,name_not_exist,not_exist_create,buyer_name,style_name
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
1,2,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,2.0,8,301.0,0.0,0.0,NaN,NaN,NaN,LEAN 4,May 4,PO38046SAND,2022-10-22 03:00:00+00:00,SA,PO 38046 Charter Hat Sand


In [22]:
dfProductionTrackingFix = dfProductionTracking.drop(columns= {'id','updated_at', 'deleted_at' , 'order_id' , 'manufacture_dept_id', 'order_id', 'prod_not_exist_order_id' })
dfProductionTrackingFix

,created_at,target_quantity,actual_quantity,complete_percent,code_order,name_order,name_cus,code_dept,name_dept,name_not_exist,not_exist_create,buyer_name,style_name
0,2022-10-22 03:00:00+00:00,927.0,318.0,34.30,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
1,2022-10-22 03:00:00+00:00,301.0,0.0,0.00,NaN,NaN,NaN,LEAN 4,May 4,PO38046SAND,2022-10-22 03:00:00+00:00,SA,PO 38046 Charter Hat Sand
2,2022-10-22 03:00:00+00:00,268.0,2.0,0.75,NaN,NaN,NaN,LEAN 5,May 5,PO38050ROL_TAN,2022-10-22 03:00:00+00:00,SA,PO38050 Kids Clear Creek Boonie
3,2022-10-22 03:00:00+00:00,289.0,0.0,0.00,NaN,NaN,NaN,LEAN 6,May 6,PO38055_ULTR_PUMI,2022-10-22 03:00:00+00:00,SA,PO38056_Ultra adventure Pumice
4,2022-10-23 03:00:00+00:00,1458.0,0.0,0.00,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702,2023-03-01 08:03:01+00:00,345.0,170.0,49.28,CZ68-221210-Recycle Plastic Visor F1,CZ68-221210-Recycle Plastic Visor F1,Intelligence Marketing CZ68,LEAN 1,May 1,NaN,NaN,NaN,NaN
1703,2023-03-01 08:12:04+00:00,994.0,145.0,14.59,TTD-221112-SP-V-S23MUF-BH73-BLACK,TTD-221112-SP-V-S23MUF-BH73-BLACK,"CapBoy Trading Co.,Ltd",LEAN 5,May 5,NaN,NaN,NaN,NaN
1704,2023-03-01 08:33:02+00:00,994.0,169.0,17.00,TTD-221112-SP-V-S23SUC-BH11-LK,TTD-221112-SP-V-S23SUC-BH11-LK,"CapBoy Trading Co.,Ltd",LEAN 4,May 4,NaN,NaN,NaN,NaN
1705,2023-03-01 10:42:02+00:00,1113.0,174.0,15.63,DYC-US-230126-Blacktiph-690-Opt03,DYC-US-230126-Blacktiph-690-Opt03,DESIGN YOUR CAPS USA LLC,LEAN 1,May 1,NaN,NaN,NaN,NaN


In [23]:
dfProductionTrackingFix = dfProductionTracking.drop(columns= {'id','updated_at', 'deleted_at' , 'order_id' , 'manufacture_dept_id', 'order_id', 'prod_not_exist_order_id' })

new_col1 = ['code_order' , 'name_order', 'name_cus' ,'created_at', 'target_quantity' ,  'actual_quantity','complete_percent', 'code_dept', 'name_dept' , 
      'name_not_exist', 'not_exist_create', 'buyer_name', 'style_name' ]
   
dfProductionTrackingFix= dfProductionTrackingFix.reindex(columns=new_col1)
   
dfProductionTrackingFix.rename(columns = { 'code_order': 'Mã hàng', 'name_order': 'Tên ĐH', 'name_cus' : 'Tên KH','created_at' : 'Ngày sản xuất' ,'target_quantity' : 'Mục tiêu' , 'actual_quantity' : 'thực tế',
                                             'complete_percent' : 'Tỉ lệ HT' , 'code_dept' : 'Mã chuyền' , 'name_dept' : 'Tên chuyền' , 'name_not_exist': 'ĐH khác' , 'not_exist_create': 'Ngày tạo'}, inplace = True)
dfProductionTrackingFix

,Mã hàng,Tên ĐH,Tên KH,Ngày sản xuất,Mục tiêu,thực tế,Tỉ lệ HT,Mã chuyền,Tên chuyền,ĐH khác,Ngày tạo,buyer_name,style_name
0,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,927.0,318.0,34.30,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
1,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,301.0,0.0,0.00,LEAN 4,May 4,PO38046SAND,2022-10-22 03:00:00+00:00,SA,PO 38046 Charter Hat Sand
2,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,268.0,2.0,0.75,LEAN 5,May 5,PO38050ROL_TAN,2022-10-22 03:00:00+00:00,SA,PO38050 Kids Clear Creek Boonie
3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,289.0,0.0,0.00,LEAN 6,May 6,PO38055_ULTR_PUMI,2022-10-22 03:00:00+00:00,SA,PO38056_Ultra adventure Pumice
4,NaN,NaN,NaN,2022-10-23 03:00:00+00:00,1458.0,0.0,0.00,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702,CZ68-221210-Recycle Plastic Visor F1,CZ68-221210-Recycle Plastic Visor F1,Intelligence Marketing CZ68,2023-03-01 08:03:01+00:00,345.0,170.0,49.28,LEAN 1,May 1,NaN,NaN,NaN,NaN
1703,TTD-221112-SP-V-S23MUF-BH73-BLACK,TTD-221112-SP-V-S23MUF-BH73-BLACK,"CapBoy Trading Co.,Ltd",2023-03-01 08:12:04+00:00,994.0,145.0,14.59,LEAN 5,May 5,NaN,NaN,NaN,NaN
1704,TTD-221112-SP-V-S23SUC-BH11-LK,TTD-221112-SP-V-S23SUC-BH11-LK,"CapBoy Trading Co.,Ltd",2023-03-01 08:33:02+00:00,994.0,169.0,17.00,LEAN 4,May 4,NaN,NaN,NaN,NaN
1705,DYC-US-230126-Blacktiph-690-Opt03,DYC-US-230126-Blacktiph-690-Opt03,DESIGN YOUR CAPS USA LLC,2023-03-01 10:42:02+00:00,1113.0,174.0,15.63,LEAN 1,May 1,NaN,NaN,NaN,NaN


In [24]:
dfProductionTrackingFix.to_excel('../dataDA/get_production_tracking.xlsx')


In [25]:
dfProductionTracking.head(5)

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,code_order,name_order,name_cus,code_dept,name_dept,name_not_exist,not_exist_create,buyer_name,style_name
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.30,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
1,2,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,2.0,8,301.0,0.0,0.00,NaN,NaN,NaN,LEAN 4,May 4,PO38046SAND,2022-10-22 03:00:00+00:00,SA,PO 38046 Charter Hat Sand
2,3,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,3.0,9,268.0,2.0,0.75,NaN,NaN,NaN,LEAN 5,May 5,PO38050ROL_TAN,2022-10-22 03:00:00+00:00,SA,PO38050 Kids Clear Creek Boonie
3,4,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,4.0,10,289.0,0.0,0.00,NaN,NaN,NaN,LEAN 6,May 6,PO38055_ULTR_PUMI,2022-10-22 03:00:00+00:00,SA,PO38056_Ultra adventure Pumice
4,5,2022-10-23 03:00:00+00:00,2022-12-15 10:01:28+00:00,NaN,NaN,1.0,7,1458.0,0.0,0.00,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet


In [26]:
dfWorkingTime = pd.read_csv('../data_csv/WorkingTime.csv')
dfProductionTrackingDetail = pd.read_csv('../data_csv/ProductionTrackingDetail.csv')
dfProductionTrackingDetail = dfProductionTrackingDetail.drop(columns= {'deleted_at' , 'updated_at'}).rename(columns = {'created_at' : 'created_detail' , 'quantity' : 'quantity_detail' ,
                                                                                                       'production_tracking_id' : 'id' ,'manufacture_dept_id' : 'manufacture_dept_id_detail' , 'target_quantity' : "target_quantity_detail" })

dfProductionTrackingDetail
dfWorkingTime = dfWorkingTime.rename(columns= {'id' : 'working_time_id' , 'code' : 'time_working'})
dfWorkingTime


,working_time_id,time_working
0,1,07:00-07:30
1,2,07:30-08:30
2,3,08:30-09:30
3,4,09:30-10:30
4,5,10:30-11:30
5,6,11:30-13:15
6,7,13:15-14:15
7,8,14:15-15:15
8,9,15:15-16:15
9,10,16:15-17:45


In [27]:
dfProdTrackingDetail = dfProductionTracking.merge(dfProductionTrackingDetail , on = ['id' , 'prod_not_exist_order_id' , 'order_id'] , how = 'right')
dfProdTrackingDetail = dfProdTrackingDetail.merge(dfWorkingTime , on = 'working_time_id' , how = 'left')
dfProdTrackingDetail

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,...,name_not_exist,not_exist_create,buyer_name,style_name,quantity_detail,created_detail,working_time_id,manufacture_dept_id_detail,target_quantity_detail,time_working
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,1,7,0.0,07:00-07:30
1,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,2,7,0.0,07:30-08:30
2,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,3,7,0.0,08:30-09:30
3,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,4,7,0.0,09:30-10:30
4,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,5,7,0.0,10:30-11:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27307,1733,2023-03-01 10:48:03+00:00,2023-03-01 13:06:06+00:00,NaN,2873.0,NaN,5,1102.0,162.0,14.7,...,NaN,NaN,NaN,NaN,0.0,2023-03-01 10:48:02+00:00,12,5,0.0,18:45-19:45
27308,1733,2023-03-01 10:48:03+00:00,2023-03-01 13:06:06+00:00,NaN,2873.0,NaN,5,1102.0,162.0,14.7,...,NaN,NaN,NaN,NaN,0.0,2023-03-01 10:48:02+00:00,13,5,0.0,19:45-20:45
27309,1733,2023-03-01 10:48:03+00:00,2023-03-01 13:06:06+00:00,NaN,2873.0,NaN,5,1102.0,162.0,14.7,...,NaN,NaN,NaN,NaN,0.0,2023-03-01 10:48:02+00:00,14,5,0.0,20:45-21:45
27310,1733,2023-03-01 10:48:03+00:00,2023-03-01 13:06:06+00:00,NaN,2873.0,NaN,5,1102.0,162.0,14.7,...,NaN,NaN,NaN,NaN,0.0,2023-03-01 10:48:02+00:00,15,5,0.0,21:45-22:00


In [28]:
dfProdTrackingDetail[(dfProdTrackingDetail['name_not_exist'] == '1666975234DH1079') & (dfProdTrackingDetail['id'] == 1)]

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,...,name_not_exist,not_exist_create,buyer_name,style_name,quantity_detail,created_detail,working_time_id,manufacture_dept_id_detail,target_quantity_detail,time_working
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,1,7,0.0,07:00-07:30
1,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,2,7,0.0,07:30-08:30
2,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,3,7,0.0,08:30-09:30
3,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,4,7,0.0,09:30-10:30
4,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,5,7,0.0,10:30-11:30
5,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,19.0,2022-10-22 03:00:00+00:00,6,7,19.0,11:30-13:15
6,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,73.0,2022-10-22 03:00:00+00:00,7,7,110.0,13:15-14:15
7,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,11.0,2022-10-22 03:00:00+00:00,8,7,110.0,14:15-15:15
8,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,215.0,2022-10-22 03:00:00+00:00,9,7,110.0,15:15-16:15
9,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,10,7,110.0,16:15-17:45


In [29]:
dfProdTrackingDetail.head(2)

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,...,name_not_exist,not_exist_create,buyer_name,style_name,quantity_detail,created_detail,working_time_id,manufacture_dept_id_detail,target_quantity_detail,time_working
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,1,7,0.0,07:00-07:30
1,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,2,7,0.0,07:30-08:30


In [30]:
dfProdTrackingDetailFix = dfProdTrackingDetail.drop(columns= {'updated_at','id', 'deleted_at' , 'order_id' , 'prod_not_exist_order_id', 'manufacture_dept_id', 'working_time_id' ,
                                                              'manufacture_dept_id_detail' , 'not_exist_create'})

dfProdTrackingDetailFix

,created_at,target_quantity,actual_quantity,complete_percent,code_order,name_order,name_cus,code_dept,name_dept,name_not_exist,buyer_name,style_name,quantity_detail,created_detail,target_quantity_detail,time_working
0,2022-10-22 03:00:00+00:00,927.0,318.0,34.3,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,07:00-07:30
1,2022-10-22 03:00:00+00:00,927.0,318.0,34.3,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,07:30-08:30
2,2022-10-22 03:00:00+00:00,927.0,318.0,34.3,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,08:30-09:30
3,2022-10-22 03:00:00+00:00,927.0,318.0,34.3,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,09:30-10:30
4,2022-10-22 03:00:00+00:00,927.0,318.0,34.3,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,10:30-11:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27307,2023-03-01 10:48:03+00:00,1102.0,162.0,14.7,DYC-US-230126-Blacktiph-690-Opt04,DYC-US-230126-Blacktiph-690-Opt04,DESIGN YOUR CAPS USA LLC,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-01 10:48:02+00:00,0.0,18:45-19:45
27308,2023-03-01 10:48:03+00:00,1102.0,162.0,14.7,DYC-US-230126-Blacktiph-690-Opt04,DYC-US-230126-Blacktiph-690-Opt04,DESIGN YOUR CAPS USA LLC,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-01 10:48:02+00:00,0.0,19:45-20:45
27309,2023-03-01 10:48:03+00:00,1102.0,162.0,14.7,DYC-US-230126-Blacktiph-690-Opt04,DYC-US-230126-Blacktiph-690-Opt04,DESIGN YOUR CAPS USA LLC,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-01 10:48:02+00:00,0.0,20:45-21:45
27310,2023-03-01 10:48:03+00:00,1102.0,162.0,14.7,DYC-US-230126-Blacktiph-690-Opt04,DYC-US-230126-Blacktiph-690-Opt04,DESIGN YOUR CAPS USA LLC,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-01 10:48:02+00:00,0.0,21:45-22:00


In [31]:
dfProdTrackingDetailFix = dfProdTrackingDetail.drop(columns= {'updated_at','id', 'deleted_at' , 'order_id' , 'prod_not_exist_order_id', 'manufacture_dept_id', 'working_time_id' ,
                                                              'manufacture_dept_id_detail' , 'not_exist_create' , 'style_name'})

new_col1 = ['code_dept' , 'name_dept' , 'code_order' , 'name_order' , 'name_cus' ,'created_at', 'time_working' ,'target_quantity_detail' , 'quantity_detail' , 'target_quantity', 'actual_quantity', 'complete_percent' , 
      'name_not_exist', 'not_exist_create', 'buyer_name', 'style_name' ]
   
dfProdTrackingDetailFix= dfProdTrackingDetailFix.reindex(columns=new_col1)
   
dfProdTrackingDetailFix.rename(columns = { 'code_dept': 'Mã chuyền', 'name_dept': 'Tên chuyền', 'code_order' : 'Mã ĐH' ,'name_order' : 'Tên ĐH', 'name_not_exist' : 'Tên ĐH không tồn tại' , 'name_cus' : 'Tên Khách hàng', 'buyer_name' : 'Tên Khách hàng khác',
                                             'created_at' : 'Ngày SX' , 'not_exist_create' : 'Ngày SX đơn hàng khác' , 'time_working' : 'Thời gian SX' , 'target_quantity_detail' : 'SL mục tiêu theo mốc TG' , 'quantity_detail': 'SL theo mốc TG' , 'target_quantity': 'SL mục tiêu tổng' , 
                                             'actual_quantity' : 'SL thực tế' , 'complete_percent' : 'Tỉ lệ hoàn thành' }, inplace = True)
dfProdTrackingDetailFix

,Mã chuyền,Tên chuyền,Mã ĐH,Tên ĐH,Tên Khách hàng,Ngày SX,Thời gian SX,SL mục tiêu theo mốc TG,SL theo mốc TG,SL mục tiêu tổng,SL thực tế,Tỉ lệ hoàn thành,Tên ĐH không tồn tại,Ngày SX đơn hàng khác,Tên Khách hàng khác,style_name
0,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,07:00-07:30,0.0,0.0,927.0,318.0,34.3,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
1,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,07:30-08:30,0.0,0.0,927.0,318.0,34.3,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
2,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,08:30-09:30,0.0,0.0,927.0,318.0,34.3,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
3,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,09:30-10:30,0.0,0.0,927.0,318.0,34.3,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
4,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,10:30-11:30,0.0,0.0,927.0,318.0,34.3,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27307,LEAN 1,May 1,DYC-US-230126-Blacktiph-690-Opt04,DYC-US-230126-Blacktiph-690-Opt04,DESIGN YOUR CAPS USA LLC,2023-03-01 10:48:03+00:00,18:45-19:45,0.0,0.0,1102.0,162.0,14.7,NaN,NaN,NaN,NaN
27308,LEAN 1,May 1,DYC-US-230126-Blacktiph-690-Opt04,DYC-US-230126-Blacktiph-690-Opt04,DESIGN YOUR CAPS USA LLC,2023-03-01 10:48:03+00:00,19:45-20:45,0.0,0.0,1102.0,162.0,14.7,NaN,NaN,NaN,NaN
27309,LEAN 1,May 1,DYC-US-230126-Blacktiph-690-Opt04,DYC-US-230126-Blacktiph-690-Opt04,DESIGN YOUR CAPS USA LLC,2023-03-01 10:48:03+00:00,20:45-21:45,0.0,0.0,1102.0,162.0,14.7,NaN,NaN,NaN,NaN
27310,LEAN 1,May 1,DYC-US-230126-Blacktiph-690-Opt04,DYC-US-230126-Blacktiph-690-Opt04,DESIGN YOUR CAPS USA LLC,2023-03-01 10:48:03+00:00,21:45-22:00,0.0,0.0,1102.0,162.0,14.7,NaN,NaN,NaN,NaN


In [16]:
dfProdTrackingDetailFix.to_excel('../dataDA/get_production_tracking_detail.xlsx')
